<a href="https://colab.research.google.com/github/AmbiTyga/Bio-VI-BERT/blob/main/Multi-label-Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title EarlyStopping
class EarlyStopping(object):
    def __init__(self, mode='min', min_delta=0, patience=10, percentage=False):
        self.mode = mode
        self.min_delta = min_delta
        self.patience = patience
        self.best = None
        self.num_bad_epochs = 0
        self.is_better = None
        self._init_is_better(mode, min_delta, percentage)

        if patience == 0:
            self.is_better = lambda a, b: True
            self.step = lambda a: False

    def step(self, metrics):
        if self.best is None:
            self.best = metrics
            return False

        if torch.isnan(metrics):
            return True

        if self.is_better(metrics, self.best):
            self.num_bad_epochs = 0
            self.best = metrics
        else:
            self.num_bad_epochs += 1

        if self.num_bad_epochs >= self.patience:
            return True

        return False

    def _init_is_better(self, mode, min_delta, percentage):
        if mode not in {'min', 'max'}:
            raise ValueError('mode ' + mode + ' is unknown!')
        if not percentage:
            if mode == 'min':
                self.is_better = lambda a, best: a < best - min_delta
            if mode == 'max':
                self.is_better = lambda a, best: a > best + min_delta
        else:
            if mode == 'min':
                self.is_better = lambda a, best: a < best - (
                            best * min_delta / 100)
            if mode == 'max':
                self.is_better = lambda a, best: a > best + (
                            best * min_delta / 100)

In [ ]:
!git clone https://github.com/rwightman/pytorch-image-models.git
%cd pytorch-image-models
!pip install -r requirements.txt -q
!python setup.py install -q
%cd ..

import os
os.kill(os.getpid(), 9)
# Restart Runtime

Cloning into 'pytorch-image-models'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 4658 (delta 29), reused 47 (delta 12), pack-reused 4573
Receiving objects: 100% (4658/4658), 15.88 MiB | 24.12 MiB/s, done.
Resolving deltas: 100% (3328/3328), done.
/content/pytorch-image-models
running install
running bdist_egg
running egg_info
creating timm.egg-info
writing timm.egg-info/PKG-INFO
writing dependency_links to timm.egg-info/dependency_links.txt
writing requirements to timm.egg-info/requires.txt
writing top-level names to timm.egg-info/top_level.txt
writing manifest file 'timm.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'timm.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/timm
copying timm/__init__.py -> build/lib/timm
copying t

In [2]:
!wget https://raw.githubusercontent.com/AmbiTyga/Bio-VI-BERT/main/Dataset.7z
!7z x /content/Dataset.7z

--2021-03-11 02:26:10--  https://raw.githubusercontent.com/AmbiTyga/Bio-VI-BERT/main/Dataset.7z
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17231696 (16M) [application/octet-stream]
Saving to: ‘Dataset.7z’

Dataset.7z          100%[===================>]  16.43M  74.2MB/s    in 0.2s    

2021-03-11 02:26:11 (74.2 MB/s) - ‘Dataset.7z’ saved [17231696/17231696]


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 17231696 bytes (17 MiB)

Extracting archive: /content/Dataset.7z
--
Path = /content/Dat

In [3]:
import timm
import pandas as pd
import numpy as np
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader, sampler
from sklearn.preprocessing import LabelEncoder
import os,time
from math import fsum
from sklearn.model_selection import train_test_split
from glob import glob
from PIL import Image
from torchvision import transforms

## Cleaning and getting a final data

In [4]:
## Getting images(file path) from the directories 

imgs = []
for path, subdirs, files in os.walk('./Dataset'):
    for name in files:
        imgs.append(os.path.join(path, name))
imgs = [x for x in imgs if '.csv' not in x]

In [64]:
data = pd.read_csv("/content/Dataset/all_meta_data.csv")
data.head()

,phylum,class,genus,species,form,sample,image_name,image_url,img_path
0,Nematoda,Chromadorea,Enterobius,Enterobius vermicularis,egg,intestinal tissue,Evermicularis_worm4_HB.jpg,https://www.cdc.gov//dpdx/enterobiasis/images/...,./Dataset/Enterobius/Evermicularis_worm4_HB.jpg
1,Nematoda,Chromadorea,Enterobius,Enterobius vermicularis,egg,intestinal tissue,Evermicularis_egg_HBa.jpg,https://www.cdc.gov//dpdx/enterobiasis/images/...,./Dataset/Enterobius/Evermicularis_egg_HBa.jpg
2,Nematoda,Chromadorea,Enterobius,Enterobius vermicularis,egg,intestinal tissue,Evermicularis_egg_wtmt.jpg,https://www.cdc.gov//dpdx/enterobiasis/images/...,./Dataset/Enterobius/Evermicularis_egg_wtmt.jpg
3,Nematoda,Chromadorea,Enterobius,Enterobius vermicularis,egg,intestinal tissue,Evermicularis_SC_egg.jpg,https://www.cdc.gov//dpdx/enterobiasis/images/...,./Dataset/Enterobius/Evermicularis_SC_egg.jpg
4,Nematoda,Chromadorea,Enterobius,Enterobius vermicularis,egg,intestinal tissue,Evermicularis_egg_UVa.jpg,https://www.cdc.gov//dpdx/enterobiasis/images/...,./Dataset/Enterobius/Evermicularis_egg_UVa.jpg


In [71]:
data['species'].value_counts()

Plasmodium vivax               36
Plasmodium falciparum          36
Plasmodium malariae            30
Cyclospora cayetanensis        30
Ascaris lumbricoides           29
Giardia duodenalis             27
Entamoeba histolytica]         27
Plasmodium ovale               27
Strongyloides stercoralis      22
Dibothriocephalus latus        21
Trichuris Trichuria            19
Enterobius vermicularis        18
Toxocara canis                 17
Ancyclostoma sp.               17
Sarcocystis sp.                16
Cryptosporidium sp.            16
Plasmodium knowlesi            15
Hymenolepis nana               14
Toxoplasma gondii              13
Trichinella sp.                13
Echinococcus granulosus        12
Fasciola hepatica              11
Hymenolepis diminuta            9
B. coli                         9
Clonorchis sinensis             8
Dicrocoelium dendriticum        7
Taenia solium                   6
Echinococcus multilocularis     4
Strongyloides fuelleborni       3
Name: species,

In [66]:
import urllib.request as req

def download(url,file_name):
  imgurl =url
  req.urlretrieve(imgurl, file_name)

In [67]:
# Dropping unnecessary datapoints
def check_file_download(x):
  if x[1] not in imgs:
    download(*x)


data['image_path_url'] = data[['image_url','img_path']].values.tolist()
data['image_path_url'] = data['image_path_url'].apply(check_file_download)

In [68]:
data.to_csv('/content/Parasitesv1.csv',index = False)

In [70]:
data

,phylum,class,genus,species,form,sample,image_name,image_url,img_path,image_path_url
0,Nematoda,Chromadorea,Enterobius,Enterobius vermicularis,egg,intestinal tissue,Evermicularis_worm4_HB.jpg,https://www.cdc.gov//dpdx/enterobiasis/images/...,./Dataset/Enterobius/Evermicularis_worm4_HB.jpg,None
1,Nematoda,Chromadorea,Enterobius,Enterobius vermicularis,egg,intestinal tissue,Evermicularis_egg_HBa.jpg,https://www.cdc.gov//dpdx/enterobiasis/images/...,./Dataset/Enterobius/Evermicularis_egg_HBa.jpg,None
2,Nematoda,Chromadorea,Enterobius,Enterobius vermicularis,egg,intestinal tissue,Evermicularis_egg_wtmt.jpg,https://www.cdc.gov//dpdx/enterobiasis/images/...,./Dataset/Enterobius/Evermicularis_egg_wtmt.jpg,None
3,Nematoda,Chromadorea,Enterobius,Enterobius vermicularis,egg,intestinal tissue,Evermicularis_SC_egg.jpg,https://www.cdc.gov//dpdx/enterobiasis/images/...,./Dataset/Enterobius/Evermicularis_SC_egg.jpg,None
4,Nematoda,Chromadorea,Enterobius,Enterobius vermicularis,egg,intestinal tissue,Evermicularis_egg_UVa.jpg,https://www.cdc.gov//dpdx/enterobiasis/images/...,./Dataset/Enterobius/Evermicularis_egg_UVa.jpg,None
...,...,...,...,...,...,...,...,...,...,...
507,Platyhelminthes,Tapeworms,Echinococcus,Echinococcus multilocularis,cyst,liver,Emultilocularis_liver_BAM2.jpg,https://www.cdc.gov//dpdx/echinococcosis/image...,./Dataset/Echinococcus/Emultilocularis_liver_B...,None
508,Platyhelminthes,Tapeworms,Echinococcus,Echinococcus multilocularis,cyst,liver,Em_cyst1,https://www.cdc.gov//dpdx/echinococcosis/image...,./Dataset/Echinococcus/Em_cyst1,None
509,Platyhelminthes,Tapeworms,Echinococcus,Echinococcus multilocularis,cyst,liver,Em_cyst_tri,https://www.cdc.gov//dpdx/echinococcosis/image...,./Dataset/Echinococcus/Em_cyst_tri,None
510,Platyhelminthes,Tapeworms,Echinococcus,Echinococcus granulosus,adult,intestinal tissue,Eg_adult_labels,https://www.cdc.gov//dpdx/echinococcosis/image...,./Dataset/Echinococcus/Eg_adult_labels,None


In [87]:
data = [x for _,x in data.groupby('species') if len(x)>12]

data = pd.concat(data,ignore_index=True)

In [73]:
labels = ["phylum","class","species","form","sample"]

In [88]:
label_encoders = dict()
for x in labels:
  le = LabelEncoder()
  data[f'{x}_label'] = le.fit_transform(data[x])
  label_encoders[x] = le

In [89]:
train, val = train_test_split(
    data, test_size=0.25, random_state=2021,stratify = data['species'])

In [90]:
train.to_csv('/content/train.csv',index = False)

val.to_csv('/content/val.csv',index = False)

# Dataset Function

In [14]:
class SpeciesDataset(Dataset):
  def __init__(self,csv_file,transform):
    super().__init__()
    csv = pd.read_csv(csv_file)[['species_label','img_path']]
    self.labels = csv['species_label'].values

    self.images = csv['img_path'].values
    self.transform = transform

    # self.LE = LabelEncoder()
    # self.labels = self.LE.fit_transform(labels)    

  def __len__(self):
    # return size of dataset
    return len(self.images)

  def __getitem__(self, index):
    img = Image.open(self.images[index])
    img = self.transform(img)

    label = self.labels[index]

    return img, torch.LongTensor([label])

In [15]:

train_transformer = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        # transforms.
        # transforms.ColorJitter(hue=.05, saturation=.05),
        transforms.RandomRotation(90),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

val_transformer = transforms.Compose([
        transforms.Resize((224,224)),
#         transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [98]:
train = pd.read_csv('/content/train.csv')
class_counts = train['species_label'].value_counts().values
weights = torch.tensor([1/class_counts[label] for label in train['species_label'].values])

train_dataset = SpeciesDataset('/content/train.csv',transform=train_transformer)
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=48,
                                           sampler=torch.utils.data.WeightedRandomSampler(weights,num_samples=540))

val = pd.read_csv('/content/val.csv')
class_counts = val['species_label'].value_counts().values
weights = torch.tensor([1/class_counts[label] for label in val['species_label'].values])

val_dataset = SpeciesDataset('/content/val.csv',transform=val_transformer)
val_loader = torch.utils.data.DataLoader(val_dataset,batch_size=17,
                                         sampler=torch.utils.data.WeightedRandomSampler(weights,num_samples=180))

# VIT Base Imagenet 21k

In [101]:
class ViT(nn.Module):
  def __init__(self,num_classes_classifier=20):
    super().__init__()
    self.img_transformer = timm.models.vision_transformer.vit_base_patch16_224_in21k(pretrained=True,num_classes = 512)
    self.drop = nn.Dropout(0.4)
    self.classifier = nn.Linear(512,num_classes_classifier)

  def forward(self,img):
    repr = self.img_transformer(img)
    repr = self.drop(repr)
    output = self.classifier(repr)

    return output  

In [103]:
model = ViT()

Removing representation layer for fine-tuning.


In [104]:
device = torch.device('cuda')
optim = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),lr = 1e-4)
loss_fn = nn.CrossEntropyLoss().to(device)
es = EarlyStopping(patience = 10)
es.best = np.inf

model.cuda()

train_losses = []
train_accs = []
val_losses = []
val_accs = []
for epoch in range(100):
  train_epoch_loss = 0
  train_epoch_acc = 0
  model.train()
  steps = 0
  for idx, batch in enumerate(train_loader):
    image, target = batch
    image = image.to(device)
    target = target.flatten().to(device)

    optim.zero_grad()

    prediction = model(image)
    # print(prediction,target)
    loss = loss_fn(prediction, target)

    num_corrects = (prediction.argmax(dim=1).view(target.size()).data == target.data).sum()
    acc = 100.0 * (num_corrects/target.size(0))
    loss.backward()
    optim.step()
    steps += 1
    
    # if steps % 6 == 0:
    #     print (f'\t Train - Epoch: {epoch+1}, Idx: {idx+1}, Training Loss: {loss.item():.4f}, Training Accuracy: {acc.item(): .2f}%')
    
    train_epoch_loss += loss.item()
    train_epoch_acc += acc.item()

  val_epoch_loss = 0
  val_epoch_acc = 0
  model.eval()
  val_steps = 0
  with torch.no_grad():
    for idx, batch in enumerate(val_loader):
      image, target = batch
      image = image.to(device)
      target = target.flatten().to(device)

      prediction = model(image)
      loss = loss_fn(prediction, target)
      
      num_corrects = (prediction.argmax(dim=1).view(target.size()).data == target.data).sum()
      acc = 100.0 * (num_corrects/target.size(0))

      val_epoch_loss += loss.item()
      val_epoch_acc += acc.item()
      val_steps +=1

  
  val_epoch_loss /=val_steps
  val_epoch_acc /=val_steps

  # val_losses.append(val_epoch_loss)
  # val_accs.append(val_epoch_acc)
  
  print(f"Epoch: {epoch+1:02}, Train Loss: {train_epoch_loss/steps:.3f}, Train Acc: {train_epoch_acc/steps:.2f}%, Val. Loss: {val_epoch_loss:.3f}, Val. Acc: {val_epoch_acc:.2f}% \n{90*'='}")
  
  if val_epoch_loss < es.best:
    best_loss = val_epoch_loss
    print(f'\r\t{es.best:.3f} --> {best_loss:.3f}  Updating')
    torch.save(model.img_transformer.state_dict(),'vit-base.bin')

    train_losses.append(train_epoch_loss/steps)
    train_accs.append(train_epoch_acc/steps)

    val_losses.append(val_epoch_loss)
    val_accs.append(val_epoch_acc)
    
  if es.step(torch.tensor(val_epoch_loss)):
    print(f'\r\tPatience complete! Best Loss is {es.best:.3f}')
    break
  if val_epoch_loss > best_loss:
    print(f'\r\t Patience -> {es.patience - es.num_bad_epochs}')

Epoch: 01, Train Loss: 2.870, Train Acc: 24.83%, Val. Loss: 2.738, Val. Acc: 29.30% 
	inf --> 2.738  Updating
Epoch: 02, Train Loss: 2.558, Train Acc: 41.84%, Val. Loss: 2.510, Val. Acc: 40.37% 
	2.738 --> 2.510  Updating
Epoch: 03, Train Loss: 2.275, Train Acc: 49.31%, Val. Loss: 2.234, Val. Acc: 41.23% 
	2.510 --> 2.234  Updating
Epoch: 04, Train Loss: 1.952, Train Acc: 59.90%, Val. Loss: 1.993, Val. Acc: 46.95% 
	2.234 --> 1.993  Updating
Epoch: 05, Train Loss: 1.653, Train Acc: 72.74%, Val. Loss: 1.779, Val. Acc: 63.96% 
	1.993 --> 1.779  Updating
Epoch: 06, Train Loss: 1.316, Train Acc: 85.42%, Val. Loss: 1.591, Val. Acc: 62.30% 
	1.779 --> 1.591  Updating
Epoch: 07, Train Loss: 1.074, Train Acc: 85.76%, Val. Loss: 1.354, Val. Acc: 73.90% 
	1.591 --> 1.354  Updating
Epoch: 08, Train Loss: 0.809, Train Acc: 93.06%, Val. Loss: 1.350, Val. Acc: 62.30% 
	1.354 --> 1.350  Updating
Epoch: 09, Train Loss: 0.616, Train Acc: 94.97%, Val. Loss: 1.170, Val. Acc: 73.05% 
	1.350 --> 1.170  Upd

# Multi-label Classification

In [105]:
class MultiDataset(Dataset):
  def __init__(self,dataFrame,label_encoders,transform=None):
    self.data = dataFrame
    self.transform = transform
    self.labels = ["phylum_label","class_label","species_label","form_label","sample_label"]
    self.label_encoder = label_encoders
    self.nclasses = self.get_nclasses()
    total = fsum(self.nclasses.values())

    self.weights = {k:v/total for k,v in self.nclasses.items()}

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    img_path = self.data.loc[idx,"img_path"]
    image = Image.open(img_path)
    image = self.transform(image)
    labels = self.data.loc[idx,self.labels].to_dict()
    for label in labels:
      labels[label] = torch.tensor(labels[label],dtype = torch.long)
            
    dict_data = {
    'image': image,
    'labels': labels
    }
    return dict_data
  
  # def get_label_encode(self):
  #   d=dict()
  #   for label in self.labels:            
  #       le = LabelEncoder()
  #       self.data[label]=le.fit_transform(self.data[label])
  #       d[label]=le
  #   return d
    
  def get_nclasses(self):
    return self.data[self.labels].nunique().to_dict()

In [106]:
class MultiViT(nn.Module):
  def __init__(self,vit_path,nclasses,labels):
    super(MultiViT,self).__init__()


    self.img_transformer = timm.models.vision_transformer.vit_base_patch16_224_in21k(pretrained=False,num_classes = 512)
    self.img_transformer.load_state_dict(torch.load(vit_path))

    for param in self.img_transformer.parameters():
      param.requires_grad = False
    self.drop = nn.Dropout(0.38)
    self.classifiers = nn.ModuleDict()
    for label in labels:
      self.classifiers[label]=nn.Linear(512,nclasses[label])

    
  def forward(self,x):
    x=self.img_transformer(x)
    x=self.drop(x)
    y=dict()
    for label in self.classifiers.keys():
        y[label]=self.classifiers[label](x)
    # print(y.keys())
    return y


In [107]:
train = pd.read_csv("/content/train.csv")
val = pd.read_csv("/content/val.csv")

train_dataset = MultiDataset(train,label_encoders,train_transformer)
val_dataset = MultiDataset(val,label_encoders,val_transformer)

train_loader = DataLoader(train_dataset,batch_size=32)
val_loader = DataLoader(val_dataset,batch_size=32)

In [108]:
multimodel = MultiViT('/content/vit-base.bin',train_dataset.nclasses,train_dataset.labels)
multimodel.to(device)
loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.Adam(filter(lambda p: p.requires_grad, multimodel.parameters()),lr = 1e-4)

Removing representation layer for fine-tuning.


In [25]:
#@title Functions for training
def get_loss(out,labels,loss_fn,weights):
    loss=0
    for label in labels:
        loss+=loss_fn(out[label],labels[label])*weights[label]
    return loss


def cal_accuracy(out,labels,batch_size,epoch_acc):
    for key in out:
        q=(out[key].detach().argmax(axis=1)==labels[key]).sum().item()/batch_size
        if not key in epoch_acc.keys():
            epoch_acc[key]=q
        else:
            epoch_acc[key]+=q
    return epoch_acc


def get_avg_acc(epoch_acc,loader):
    acc=0
    for key in epoch_acc:
        epoch_acc[key]/=len(loader)
        acc+=epoch_acc[key]
        
    return acc/len(epoch_acc),epoch_acc

def evaluate(model,loss_fn,loader):
    model.eval()
    epoch_acc={}
    epoch_loss=0
    for batch in loader:
        img=batch["image"].to(device)
        labels=batch["labels"]
        for key in labels:
            labels[key]=labels[key].to(device)
        
        out=model(img)
        loss=get_loss(out,labels,loss_fn,loader.dataset.weights)
        epoch_loss+=loss.detach().item()
        epoch_acc=cal_accuracy(out,labels,img.shape[0],epoch_acc)
        
    avg_acc,epoch_acc=get_avg_acc(epoch_acc,loader)
    return (epoch_loss/len(loader),avg_acc,epoch_acc)

def train_model(model, loss_fn, opt,trainloader,valloader, num_epochs=1):
    since = time.time()
    best_model=""
    max_loss=np.inf
    train={"loss":[],"avg":[],"individual":[]}
    val={"loss":[],"avg":[],"individual":[]}
    for epoch in range(num_epochs):
        print('Epoch {} of {}'.format(epoch+1, num_epochs))
        epoch_loss=0
        epoch_acc={}
        model.train()
        # Iterate over data.
        for i,data in enumerate(trainloader):
            opt.zero_grad()
            img=data["image"].to(device)
            labels=data["labels"]
            for key in labels:
                labels[key]=labels[key].to(device)
            

            out=model(img)
            # print(out,labels,sep = '\n\n')
            loss=get_loss(out,labels,loss_fn,trainloader.dataset.weights)
            epoch_loss+=loss.detach().item()
            loss.backward()
            opt.step()
            epoch_acc=cal_accuracy(out,labels,img.shape[0],epoch_acc)
            
        val_result=evaluate(model,loss_fn,valloader)
        avg_acc,epoch_acc=get_avg_acc(epoch_acc,trainloader)
        avg_loss=epoch_loss/len(trainloader)
        print('Train')
        print(f"Loss: {avg_loss}\nAccuracy: Each_label_acc->{epoch_acc}\n\tAvg acc->{avg_acc}\n")
        print("Validation")
        print(f"Loss: {val_result[0]}\nAccuracy: Each_label_acc->{val_result[2]}\n\tAvg acc->{val_result[1]}")
        print("-"*50)
        
        train["avg"].append(avg_acc)
        train["individual"].append(epoch_acc)
        train["loss"].append(avg_loss)
        
        val["avg"].append(val_result[1])
        val["individual"].append(val_result[2])
        val["loss"].append(val_result[0])
        if val_result[0]<max_loss:
            max_loss=val_result[0]
            torch.save(model.state_dict(),'multimodal.bin')

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    return train,val,best_model,max_loss

In [116]:
train,val,best_model,best_acc=train_model(multimodel, loss_fn, optim,train_loader,val_loader,num_epochs=140)

Epoch 1 of 140
Train
Loss: 0.323371403596618
Accuracy: Each_label_acc->{'phylum_label': 1.0, 'class_label': 1.0, 'species_label': 1.0, 'form_label': 0.634469696969697, 'sample_label': 0.824810606060606}
	Avg acc->0.8918560606060606

Validation
Loss: 0.7545453906059265
Accuracy: Each_label_acc->{'phylum_label': 0.9279279279279279, 'class_label': 0.8918918918918919, 'species_label': 0.7747747747747747, 'form_label': 0.5495495495495496, 'sample_label': 0.7837837837837838}
	Avg acc->0.7855855855855857
--------------------------------------------------
Epoch 2 of 140
Train
Loss: 0.3258802457289262
Accuracy: Each_label_acc->{'phylum_label': 1.0, 'class_label': 1.0, 'species_label': 1.0, 'form_label': 0.6231060606060607, 'sample_label': 0.8267045454545454}
	Avg acc->0.8899621212121211

Validation
Loss: 0.7542915344238281
Accuracy: Each_label_acc->{'phylum_label': 0.9279279279279279, 'class_label': 0.8918918918918919, 'species_label': 0.7747747747747747, 'form_label': 0.5495495495495496, 'samp

In [117]:
multimodel.load_state_dict(torch.load('/content/multimodal.bin'))
multimodel.to(device)
multimodel.eval()
val_loader = DataLoader(val_dataset,batch_size=len(val_dataset))

target = next(iter(val_loader))
img,labels = target['image'], target['labels']

img = img.to(device)
pred = multimodel(img)

In [118]:

for x in pred:

  pred[x] = pred[x].cpu().detach().argmax(axis = 1).numpy()
  labels[x] = labels[x].numpy()

In [119]:
from sklearn.metrics import classification_report, confusion_matrix
y = '_label'
for label in pred:
  label_le = label.replace(y,'')
  classes = val_loader.dataset.label_encoder[label_le].classes_
  print(f"\nClassification Report for {label}:\n")
  print(classification_report(labels[label],pred[label],target_names=classes))
  print('='*95)


Classification Report for phylum_label:

                   precision    recall  f1-score   support

        Amoebozoa       1.00      0.86      0.92         7
      Apicomplexa       0.96      0.95      0.95        56
         Nematoda       0.89      1.00      0.94        33
  Platyhelminthes       0.83      0.62      0.71         8
Sarcomastigophora       0.86      0.86      0.86         7

         accuracy                           0.93       111
        macro avg       0.91      0.86      0.88       111
     weighted avg       0.93      0.93      0.93       111


Classification Report for class_label:

               precision    recall  f1-score   support

 Aconoidasida       1.00      1.00      1.00        37
      Cestoda       0.62      0.62      0.62         8
  Chromadorea       0.81      0.88      0.85        25
  Conoidasida       0.82      0.95      0.88        19
      Enoplea       1.00      0.75      0.86         8
    Tubulinea       1.00      0.71      0.83        

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ValueError: ignored

In [ ]:
pred

{'class': tensor([2, 5, 0, 3, 2, 3, 1, 0, 5, 2, 6, 2, 3, 0, 0, 0, 5, 2, 3, 3, 2, 3, 2, 0,
         0, 1, 6, 6, 4, 0, 0, 6, 2, 0, 2, 0, 2, 3, 2, 0, 3, 0, 5, 0, 0, 2, 0, 2,
         0, 3, 0]),
 'form': tensor([ 3,  1, 11,  6,  0,  6,  3, 11,  6,  0,  1,  0,  6, 11, 11, 11, 11,  0,
          6,  6,  0,  3,  3, 11, 11,  3,  1,  1,  3,  8, 11,  1,  5,  8,  3, 10,
          5,  6,  3, 11,  6, 11, 11, 11, 11,  5, 11,  0, 11,  6, 11]),
 'phylum': tensor([2, 0, 1, 2, 2, 1, 3, 1, 0, 2, 4, 2, 1, 1, 1, 1, 0, 2, 1, 1, 2, 2, 2, 1,
         1, 3, 4, 4, 2, 1, 1, 4, 2, 1, 2, 1, 2, 1, 2, 1, 1, 1, 0, 1, 1, 2, 1, 2,
         1, 1, 1]),
 'sample': tensor([1, 9, 0, 9, 3, 6, 1, 0, 9, 3, 9, 1, 1, 0, 0, 0, 9, 1, 1, 1, 2, 1, 1, 0,
         0, 1, 9, 9, 2, 0, 0, 9, 1, 0, 1, 0, 1, 1, 2, 0, 1, 0, 9, 0, 0, 1, 0, 1,
         0, 1, 0]),
 'species': tensor([ 1,  5, 11, 12, 14, 12,  4, 10,  5, 14,  7,  1,  3, 11, 10, 11,  5,  1,
          3,  3,  6, 12,  1,  8, 11,  4,  7,  7, 15,  8, 10,  7, 13,  8,  1,  9,
          0

In [62]:
pd.read_csv('train.csv').shape, pd.read_csv('val.csv').shape

((291, 15), (97, 15))

In [63]:
291+97

388